# <center> Bài tập Thực hành môn Xử lý Đa chiều </center>

### **Họ và tên:** Huỳnh Nguyễn Thế Dân
### **MSSV:** 21110256
### **Lớp:** 21TTH1

# <center> Trả lời </center>

## Bài 2.

In [2]:
import numpy as np

# a) Phát sinh tập dữ liệu D gồm 1000 điểm có phân phối đều U(-10; 10) với số chiều là 20 và 1 điểm dữ liệu truy vấn q
def generate_data(num_points, dimensions, lower_bound, upper_bound):
    data = np.random.uniform(lower_bound, upper_bound, (num_points, dimensions))
    query_point = np.random.uniform(lower_bound, upper_bound, dimensions)
    return data, query_point

# Sử dụng hàm generate_data để tạo tập dữ liệu D và điểm truy vấn q
D, q = generate_data(1000, 20, -10, 10)


In [3]:
# b) Viết hàm generate_random_hyperplanes(num_planes, dimensions) để phát sinh các siêu phẳng ngẫu nhiên
def generate_random_hyperplanes(num_planes, dimensions):
    return np.random.randn(num_planes, dimensions)

# Áp dụng hàm này để phát sinh 10 siêu phẳng có số chiều 20
hyperplanes = generate_random_hyperplanes(10, 20)


In [4]:
# c) Viết hàm lsh_hash_points(points, hyperplanes) để mã hoá các điểm dữ liệu
def lsh_hash_points(points, hyperplanes):
    projections = np.dot(points, hyperplanes.T)
    return (projections > 0).astype(int)

# Áp dụng hàm này để mã hoá tập dữ liệu D trong câu a)
hashed_D = lsh_hash_points(D, hyperplanes)


In [5]:
from scipy.spatial.distance import hamming

# d) Viết hàm query_lsh(hash_table, query_point, hyperplanes) để mã hoá điểm dữ liệu cần truy vấn
def query_lsh(hash_table, query_point, hyperplanes):
    query_hash = lsh_hash_points(query_point.reshape(1, -1), hyperplanes).flatten()
    hamming_distances = np.array([hamming(query_hash, h) for h in hash_table])
    return np.where(hamming_distances == hamming_distances.min())[0], hamming_distances.min()

# Áp dụng hàm này để tìm các điểm có mã tương đồng với điểm q trong câu a)
similar_points_indices, min_hamming_distance = query_lsh(hashed_D, q, hyperplanes)


In [6]:
from scipy.spatial.distance import euclidean

# e) Trong những điểm tìm được ở câu d), điểm nào tương đồng với q nhất (theo khoảng cách Euclide)?
def find_most_similar(data, query_point, indices):
    min_distance = float('inf')
    most_similar_point = None
    for idx in indices:
        dist = euclidean(data[idx], query_point)
        if dist < min_distance:
            min_distance = dist
            most_similar_point = data[idx]
    return most_similar_point, min_distance

most_similar_point, min_distance = find_most_similar(D, q, similar_points_indices)


In [7]:
# f) Sử dụng thuật toán nearest neighbors thông thường để tìm điểm dữ liệu trong D gần q nhất và so sánh với kết quả ở câu e)
def find_nearest_neighbor(data, query_point):
    distances = np.linalg.norm(data - query_point, axis=1)
    nearest_neighbor_idx = np.argmin(distances)
    return data[nearest_neighbor_idx], distances[nearest_neighbor_idx]

nearest_neighbor, nn_distance = find_nearest_neighbor(D, q)

# So sánh kết quả
print("Most similar point using LSH: ", most_similar_point, "with Euclidean distance: ", min_distance)
print("Nearest neighbor using conventional method: ", nearest_neighbor, "with Euclidean distance: ", nn_distance)


Most similar point using LSH:  [-2.04646703 -5.9162225  -3.65568223 -2.49252596  7.97476041  1.42333095
  1.2573416  -9.12329523  3.31071439 -3.09588071  3.82211002 -7.48359999
 -7.2088187  -5.94132428  3.20285412  9.03044113 -0.69548912  5.46095148
 -4.00535011  5.60775353] with Euclidean distance:  22.5180401960357
Nearest neighbor using conventional method:  [-2.04646703 -5.9162225  -3.65568223 -2.49252596  7.97476041  1.42333095
  1.2573416  -9.12329523  3.31071439 -3.09588071  3.82211002 -7.48359999
 -7.2088187  -5.94132428  3.20285412  9.03044113 -0.69548912  5.46095148
 -4.00535011  5.60775353] with Euclidean distance:  22.518040196035702


In [9]:
# g) Chạy lại các bước trên nhiều lần và nhận xét
def run_experiment(num_runs):
    lsh_distances = []
    nn_distances = []
    for _ in range(num_runs):
        D, q = generate_data(1000, 20, -10, 10)
        hyperplanes = generate_random_hyperplanes(10, 20)
        hashed_D = lsh_hash_points(D, hyperplanes)
        similar_points_indices, _ = query_lsh(hashed_D, q, hyperplanes)
        _, lsh_min_distance = find_most_similar(D, q, similar_points_indices)
        _, nn_distance = find_nearest_neighbor(D, q)
        lsh_distances.append(lsh_min_distance)
        nn_distances.append(nn_distance)
    return lsh_distances, nn_distances

lsh_distances, nn_distances = run_experiment(10)

print("LSH distances: ", lsh_distances)
print("Nearest neighbor distances: ", nn_distances)


LSH distances:  [30.485843104590653, 26.82143284221735, 24.64359678376673, 26.520599028165694, 31.63287419389769, 21.07837581064271, 28.842786007192608, 23.717362275613272, 32.63107712963388, 30.4220277896504]
Nearest neighbor distances:  [22.678763891454775, 19.465200350180904, 21.557176224881264, 22.16515701531989, 22.49960821748461, 20.16155335483088, 21.06984033372809, 18.712881528540006, 18.413843709727004, 20.13684582431757]


### Nhận xét: 
- LSH giúp giảm đáng kể số lượng phép tính khoảng cách cần thiết, mặc dù kết quả có thể không chính xác bằng phương pháp tìm hàng xóm gần nhất thông thường.
- Kết quả của LSH phụ thuộc vào việc phát sinh ngẫu nhiên các siêu phẳng, do đó có thể thay đổi khi chạy lại nhiều lần.


----------------------------------------------------------------

### <center> Hết </center>